In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import pandas as pd

from sklearn.metrics import accuracy_score

from scipy.sparse import hstack
from sklearn.preprocessing import LabelEncoder

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import confusion_matrix

import requests
import json

Generate test and training sets - adicionar falas de "outros"

In [8]:
# Load the CSV file into a Pandas DataFrame
df = pd.read_csv('friends.csv', index_col=False)

# Split the dataset into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Create the training data with only 'character' and 'line' columns
train_data = train_df[['character', 'line']]

# Create the testing data with only the 'line' column
test_data = test_df[['character', 'line']]

create naive-bayes classifier

In [9]:
# Define the labels
labels = ["Rachel", "Monica", "Joey", "Chandler", "Ross", "Phoebe"]

# Word Counts
vectorizer = CountVectorizer()
train_vectors = vectorizer.fit_transform(train_data['line'])
test_vectors = vectorizer.transform(test_data['line'])

print(train_data['line'])

# Train the Naive Bayes classifier
clf = MultinomialNB()
clf.fit(train_vectors, train_data['character'])

# Classify the test set
test_predictions = clf.predict(test_vectors)

# Print the predicted labels for the test set
i=0
for index, row in test_data.iterrows():
    #print("Line {}: {}".format(i+1, test_data['line'][i]))
    print("Line {}: {}".format(i+1, row))
    print("Predicted character: {}\n".format(test_predictions[i]))
    i+=1

19315                                               What?!
29076     All right, it won’t go down any further. It’s...
17011     So I'm a pimp huh? It's okay! Look, I know th...
12858            Well, he gets the other one all riled up.
37368                                              Daddy?!
                               ...                        
11284     Great, actually. I'm thinking tonight, maybe ...
44732                     Well it stupid, unfair question!
38158                                 No thanks, I�m good.
860               Alright. (on phone) Yes, this is Rachel.
15795     Rach, that's great! It's so good that you had...
Name: line, Length: 37440, dtype: object
Line 1: character      Ross
line          What?
Name: 37454, dtype: object
Predicted character: Rachel

Line 2: character                                       Phoebe
line          Well, didn’t you just hear what I said?!
Name: 27552, dtype: object
Predicted character: Rachel

Line 3: character               

In [139]:
# Calculate the accuracy of the predictions
accuracy = accuracy_score(test_labels, test_predictions)

print("Accuracy: {:.2f}%".format(accuracy*100))

# Calculate the confusion matrix for the predictions
confusion_mat = confusion_matrix(test_labels, test_predictions)
print(confusion_mat)

# Compute evaluation metrics using the confusion matrix
accuracy = (confusion_mat[0,0] + confusion_mat[1,1]) / len(y_test)
precision = confusion_mat[1,1] / (confusion_mat[1,1] + confusion_mat[0,1])
recall = confusion_mat[1,1] / (confusion_mat[1,1] + confusion_mat[1,0])
f1_score = 2 * precision * recall / (precision + recall)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1_score)

Accuracy: 29.64%
[[321 254 207  88 421 254]
 [118 518 167  78 418 285]
 [167 167 368  93 464 264]
 [120 214 163 251 440 204]
 [132 194 168 125 827 230]
 [155 280 167  99 462 494]]
Accuracy: 0.08956976620049109
Precision: 0.6709844559585493
Recall: 0.8144654088050315
F1-score: 0.7357954545454546


Feature set:
-  N-grams

In [111]:
# Vectorize the text data using trigrams
vectorizer = CountVectorizer(ngram_range=(1, 5))
train_vectors = vectorizer.fit_transform(train_data['line'])
test_vectors = vectorizer.transform(test_data['line'])

# Train the Naive Bayes classifier
clf = MultinomialNB()
clf.fit(train_vectors, train_data['character'])

# Classify the test set
test_predictions = clf.predict(test_vectors)

# Calculate the accuracy of the predictions
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_data['character'], test_predictions)
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 30.07%


Feature set:
-  word frequency counts
- N-grams

In [112]:
# Vectorize the text data using word frequencies and n-grams
vectorizer1 = CountVectorizer()
vectorizer2 = CountVectorizer(ngram_range=(1, 5), analyzer='char')
X1_train = vectorizer1.fit_transform(train_data['line'])
X2_train = vectorizer2.fit_transform(train_data['line'])
X_train = hstack([X1_train, X2_train])
X1_test = vectorizer1.transform(test_data['line'])
X2_test = vectorizer2.transform(test_data['line'])
X_test = hstack([X1_test, X2_test])

# Encode the class labels
le = LabelEncoder()
y_train = le.fit_transform(train_data['character'])
y_test = le.transform(test_data['character'])

# Train the Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train, y_train)

# Classify the test set
test_predictions = clf.predict(X_test)

# Decode the predicted class labels
predicted_labels = le.inverse_transform(test_predictions)

# Calculate the accuracy of the predictions
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_data['character'], predicted_labels)
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 31.09%


Feature Set:
- TF-IDF

In [113]:
# Vectorize the text data using TF-IDF
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_data['line'])
X_test = vectorizer.transform(test_data['line'])

# Encode the class labels
le = LabelEncoder()
y_train = le.fit_transform(train_data['character'])
y_test = le.transform(test_data['character'])

# Train the Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train, y_train)

# Classify the test set
test_predictions = clf.predict(X_test)

# Decode the predicted class labels
predicted_labels = le.inverse_transform(test_predictions)

# Calculate the accuracy of the predictions
accuracy = accuracy_score(test_data['character'], predicted_labels)
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 28.23%


Feature set:
-  word frequency counts
- TF-IDF scores
- word count
- character count
- average word length

In [121]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# extract features
# word frequency counts
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(train_data['line'])
X_test_counts = count_vect.transform(test_data['line'])

# TF-IDF scores
tfidf_vect = TfidfVectorizer()
X_train_tfidf = tfidf_vect.fit_transform(train_data['line'])
X_test_tfidf = tfidf_vect.transform(test_data['line'])

# word count
train_data['word_count'] = train_data['line'].apply(lambda x: len(str(x).split(" ")))
test_data['word_count'] = test_data['line'].apply(lambda x: len(str(x).split(" ")))

# character count
train_data['char_count'] = train_data['line'].str.len()
test_data['char_count'] = test_data['line'].str.len()

# average word length
def avg_word_len(sentence):
    words = sentence.split()
    return sum(len(word) for word in words)/len(words)

train_data['avg_word_len'] = train_data['line'].apply(lambda x: avg_word_len(x))
test_data['avg_word_len'] = test_data['line'].apply(lambda x: avg_word_len(x))

# concatenate features into a single feature matrix
X_train_features = np.concatenate((X_train_counts.toarray(), 
                                    X_train_tfidf.toarray(),
                                    train_data[['word_count', 'char_count', 'avg_word_len']].values), axis=1)

X_test_features = np.concatenate((X_test_counts.toarray(),
                                    X_test_tfidf.toarray(),
                                    test_data[['word_count', 'char_count', 'avg_word_len']].values), axis=1)

# create target variables
y_train = train_data['character']
y_test = test_data['character']

# train and test classifier
nb = make_pipeline(StandardScaler(), MultinomialNB())
nb.fit(X_train_features, y_train)
accuracy = nb.score(X_test_features, y_test)

TypeError: 'str' object is not callable

contexto (adicionar frases em conj)

test set por season/seaons

matrizes de confusão - perceber que pares de personagem é que "confudem" mais o classificador